In [ ]:
dir_root = '/user/centos/'
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [ ]:
from pyspark.sql import functions as sf

# DataFrames

## 0. Why Spark dataframes?

- #### Schema (strongly typed)

- #### Optimizations via Catalyst
    - like SQL query planner + physical plan
    - more declarative
    - e.g. filter push down


* #### SPEED

<img src="images/dataframe-api-speed.png" width="80%" />

- #### Familiar to SQL users
    - similar expressions

- #### Simpler code

*RDDs*
```python
(rdd.map(lambda x: (x[0], (x[1], 1)))
    .reduceByKey(sum_pair)
    .mapValues(lambda s: s[0] / s[1]))
```

Where

```python    
def sum_pair(pair):
    x, y = pair
    return (x[0] + y[0], x[1] + y[1])
```

*DataFrame API*
```python
(ddf.groupBy('name')
    .agg({'age': 'avg'})
    .collect())
```

## 1. Getting data

### 1.1 Create

#### From rdd to dataframe

In [ ]:
rdd = sc.parallelize([[None, 'Michael'],
                      [30, 'Andy'],
                      [19, 'Justin'],
                      [30, 'James Dr No From Russia with Love Bond']])
rdd.collect()

In [ ]:
rdd.toDF()

In [ ]:
ddf = rdd.toDF() 
ddf.show()

#### Directly from python

In [ ]:
ddf_names = spark.createDataFrame([[None, 'Michael'],
                             [30, 'Andy'],
                             [19, 'Justin'],
                             [30, 'James Dr No From Russia with Love Bond']], 
                            schema = ['age', 'name'])

In [ ]:
ddf_names.show()

In [ ]:
ddf_names.columns

In [ ]:
ddf_names.dtypes

In [ ]:
ddf_names.schema

In [ ]:
ddf_names.printSchema()

#### Still an rdd

In [ ]:
ddf_names.first()

In [ ]:
ddf_names.rdd.map(lambda r: r['age'] + 1 if r['age'] != None else r['age']).collect()

#### If your name is too long

Mr. David Feirn

```James Dr No From Russia with Love Goldfinger Thunderball You Only Live Twice On Her Majestys Secret Service Diamonds Are Forever Live and Let Die The Man with the Golden Gun The Spy Who Loved Me Moonraker For Your Eyes Only Octopussy A View to a Kill The Living Daylights Licence to Kill Golden Eye Tomorrow Never Dies The World Is Not Enough Die Another Day Casino Royale Bond```

In [ ]:
ddf_names.show()

In [ ]:
ddf_names.show(n=10, truncate=False)

### 1.2 Importing data

In [ ]:
local_prepend = 'file://'
hdfs_prepend = 'hdfs://'

#### CSV

In [ ]:
spark.read.csv(dir_root + 'data/heroes.csv').show(3)

In [ ]:
ddf_heroes = spark.read.csv(dir_root + 'data/heroes.csv', header=True)
ddf_heroes.show(3)

In [ ]:
ddf_heroes.show(20)

In [ ]:
ddf_heroes.columns

In [ ]:
[c for c in ddf_heroes.columns if c != '_c0']

In [ ]:
ddf_heroes.select([c for c in ddf_heroes.columns if c != '_c0'])

#### Pandas

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/notebooks/data/heroes.csv', index_col = 0)

In [ ]:
ddf = spark.createDataFrame(df)
ddf.show(3)

In [ ]:
ddf.toPandas().head(3)

#### Parquet
- preferred format
    - small file size (efficient compression)
    - schema
    - works accross machines (unlike pickle)
   

In [ ]:
ddf_airlines = spark.read.parquet(dir_root + 'data/airlines.parquet')

#### Other formats

* Jdbc;
* HDFS;
* Avro*;
* HBase*;
* Cassandra*;
* etc.

${}^*$ external

## 2. Basics (do stuff)
- `filter`
- `select`
- `sort`
- `groupBy`

### 2.1 select()
- select columns
- make new columns

#### Selecting existing cols

In [ ]:
from pyspark.sql import functions as sf # Spark functions

In [ ]:
(ddf_names.select('name', # if possible
            ddf_names.name, # no
            ddf_names['name'], # no
            sf.col('name')) # all other cases
    .show())

#### Create new columns

In [ ]:
ddf_names.age

In [ ]:
ddf_names['age']

In [ ]:
(ddf_names.select('*',
            ddf_names['age'] + 1,
            sf.sqrt('age')) # apply Spark functionf
    .show())

In [ ]:
ddf_names2 = ddf_names.select(ddf_names.age + 1)
ddf_names2.columns

In [ ]:
(ddf_names2.select('(age + 1)')
     .show())

#### Hive

In [ ]:
ddf_names.registerTempTable('ddf_names')

In [ ]:
ddf_names_sql = spark.sql("""
SELECT age FROM ddf_names
""").show()

#### Naming new columns

In [ ]:
(ddf_names.select((ddf_names.age + 1).alias('age_inc')) 
    .show())

In [ ]:
ddf_names.select('*', 'age')

In [ ]:
(ddf_names.withColumn('age_inc', ddf_names.age + 1)).cache()

#### *Intermezzo: query formatting*

Pythonista's
```python
(
    ddf.groupBy('name')
    .agg({'age': 'avg'})
    .collect()
)
```

Some others
```python
(ddf.groupBy('name')
    .agg({'age': 'avg'})
    .collect())
```

Another alternative
```python
(ddf
 .groupBy('name')
 .agg({'age': 'avg'})
 .collect())
```

### 2.2 sort()

In [ ]:
(ddf_names
 .sort('age')
 .show())

In [ ]:
(ddf_names
 .sort(sf.col('age').desc())
 .show())

### 2.3 filter()

In [ ]:
(ddf_names
 .filter(sf.col('age') > 21)
 .show())

In [ ]:
(ddf_names
 .filter((sf.col('age') > 21) &
         (sf.col('name') != 'Andy'))
 .show())

### 2.4 groupBy() -> agg() (aggregate)

In [ ]:
(ddf_names
 .groupBy("age")
 .count()
 .show())

In [ ]:
(ddf_names
 .groupBy("age")
 .agg({'age': 'max', 'age': 'first', 'age': 'stddev'}) # why does this happen?
 .show())

In [ ]:
{'age': 'max', 'age': 'first', 'age': 'stddev'}

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [ ]:
schema = StructType([StructField('age', StringType()), StructField('name', StringType())])

In [ ]:
ddf_names_str = spark.createDataFrame(ddf_names.rdd, schema = schema)
ddf_names_str.printSchema()

In [ ]:
cols = [sf.first(c).alias(c + '_agg') for c in ['age', 'name']]

In [ ]:
(ddf_names_str
 .groupBy("age")
 .agg(sf.max('age').alias('max_age'),
      sf.first('age').alias('first_age'),
      sf.stddev('age').alias('stddev_age'), 
      *cols)
 .show())

In [ ]:
ddf_names_str.withColumn('age_int',sf.col('age').cast('Int'))

### *Exercises*

1. Load the Heroes of the Storm dataset with read csv ('data/heroes.csv' in the hdfs homedir of user centos)
2. check the dtypes: 
    - what do you notice?
    - Fix it.
2. Manually explore the data and look for corrupted/malformed data
3. Find a way to remove these corrupted rows
3. Which hero has the most hp?
4. Add a column with the 'attack_momentum', computed as attack * attack_spd
5. Which role on average has the highest attack?
6. Figure out which roles and attack_type frequently co-occur
7. Deliver a dataframe with the highest attack per role 
8. export to Pandas

Bonus
9. make a function that accepts a dataframe and a list colnames. Let it return the mean and stddev of the columns 
10. apply the function to the hp and attack column such that the result has columns:

`hp_mean, hp_stddev, attack_mean, attack_stddev`